In [83]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)


In [84]:
df = pd.read_csv("../../datos/02_api_rent_limpieza_cols.csv")
df.sample()

,propertyType,status,price,size,rooms,bathrooms,floor,neighborhood,address,district,province,municipality,exterior,hasLift,hasPlan,has3DTour,has360,distance
294,flat,good,550.00,76.00,2,1,2,NaN,Yuncos,NaN,Toledo,Yuncos,True,True,False,False,False,38544


# Gestión de Valores erróneos
Vamos a revisarlos en:
- Precio
- pricebyarea
- size
- rooms
- bathrooms


In [85]:
df["province"].unique()

array(['Toledo', 'Madrid', 'Guadalajara', 'Segovia', 'Ávila'],
      dtype=object)

### Modelo 7 Eliminar casas ajenas a "Madrid" 

In [86]:
df = df[df["province"] == "Madrid"]
df.reset_index(drop=True,inplace=True)
df.shape

(353, 18)

In [87]:
df["price"].unique()

array([750., 400., 590., 684., 600., 550., 700., 695., 650., 675., 680.,
       747., 640., 625., 720., 699., 620., 500., 595., 725., 666., 630.,
       635., 690., 730., 624., 633., 645., 735., 705., 710., 610., 580.,
       560., 740., 670., 723., 450., 660., 692., 609., 718., 694., 658.,
       728., 715., 475., 749., 667., 525., 733., 745.])

No se ven anómalos

### size

In [88]:
df["size"].nlargest(5)

18    206.00
248   180.00
195   168.00
225   161.00
191   155.00
Name: size, dtype: float64

In [89]:
df["size"].nsmallest(5)

280   20.00
137   23.00
236   23.00
27    25.00
108   25.00
Name: size, dtype: float64

No parece erróneos

### rooms

In [90]:
df["rooms"].unique()

array([1, 2, 3, 0, 4])

0 es que son estudios

### bathrooms

In [91]:
df["bathrooms"].unique()

array([1, 2, 3])

No se ven valores erróneos

# Corregir "floor"
- Vamos a pasar todo a texto para que sea una categórica

In [92]:
df["floor"].unique()

array(['3', 'bj', '2', nan, '1', '5', 'en', '4', 'st', '8', '7', '6',
       '14', 'ss'], dtype=object)

### Generamos un diccionario para aplicar los cambios

In [93]:
diccionario_pisos = {
    "1": "primero",
    "2": "segundo",
    "3": "tercero",
    "4": "cuarto",
    "5": "quinto",
    "6": "sexto",
    "7": "septimo",
    "8": "octavo",
    "14": "decimo cuarto",
    "bj": "bajo",
    "en": "entreplanta",
    "ss": "sotano",
    "st": "sotano",
}

df["floor"] = df["floor"].map(diccionario_pisos)
df["floor"].unique()


array(['tercero', 'bajo', 'segundo', nan, 'primero', 'quinto',
       'entreplanta', 'cuarto', 'sotano', 'octavo', 'septimo', 'sexto',
       'decimo cuarto'], dtype=object)

### Modelo 13 Recategorizar pisos

In [94]:
bajos = ["primero","sotano","bajo","entreplanta"]
altos = ["segundo","tercero","cuarto","quinto","sexto","septimo","octavo","decimo cuarto"]

In [95]:
df.loc[df["floor"].isin(bajos), "floor"] = "bajos"
df.loc[df["floor"].isin(altos), "floor"] = "altos"

In [96]:
df["floor"].value_counts()

floor
altos    154
bajos    136
Name: count, dtype: int64

# Corregir Bathrooms
- Vamos a pasar todo a texto para que sea una categórica

In [97]:
df["bathrooms"].unique()

array([1, 2, 3])

In [98]:
df.loc[df["bathrooms"] == 1, "bathrooms"] = "1 aseo"
df.loc[df["bathrooms"] == 2, "bathrooms"] = "2 aseos"
df.loc[df["bathrooms"] == 3, "bathrooms"] = "3 aseos"

In [99]:
df["bathrooms"].unique()

array(['1 aseo', '2 aseos', '3 aseos'], dtype=object)

# Corregir rooms
- Vamos a pasar todo a texto para que sea una categórica

In [100]:
df["rooms"].unique()

array([1, 2, 3, 0, 4])

In [101]:
df.loc[df["rooms"] == 0, "rooms"] = "sin habitaciones"
df.loc[df["rooms"] == 1, "rooms"] = "1 habitacion"
df.loc[df["rooms"] == 2, "rooms"] = "2 habitaciones"
df.loc[df["rooms"] == 3, "rooms"] = "3 habitaciones"
df.loc[df["rooms"] == 4, "rooms"] = "4 habitaciones"
df.loc[df["rooms"] == 6, "rooms"] = "6 habitaciones"

In [102]:
df["rooms"].unique()

array(['1 habitacion', '2 habitaciones', '3 habitaciones',
       'sin habitaciones', '4 habitaciones'], dtype=object)

# Reducción / Simplificación de columnas
- Vamos a quitarnos todo lo que:
    - Especifique demasiado para el precio
- Vamos a simplificar todo lo que:
    - Especifique demasiado, pero al generalizarlo, aporte valor

In [103]:
df.sample()

,propertyType,status,price,size,rooms,bathrooms,floor,neighborhood,address,district,province,municipality,exterior,hasLift,hasPlan,has3DTour,has360,distance
200,flat,good,590.00,52.00,1 habitacion,1 aseo,bajos,NaN,Los Molinos,NaN,Madrid,Los Molinos,True,False,False,False,False,44526


### Dropeamos
- address: No se puede generalizar

In [104]:
df.drop(columns="address",inplace=True)

### Generalizamos
- distance: Lo haremos por rangos de distancia al centro

In [105]:
df["distance"].nlargest()

3      56596
2      55041
188    53247
186    53238
289    49280
Name: distance, dtype: int64

In [106]:
df["distance"].nsmallest()

108    183
126    470
27     533
136    625
317    691
Name: distance, dtype: int64

In [107]:
df["distancia_centro"] = "unknown"
df.loc[df["distance"].between(0,1000,inclusive="left"), "distancia_centro"] = "Menos de 1 km"
df.loc[df["distance"].between(1000,5000,inclusive="left"), "distancia_centro"] = "Entre 1 y 5 km"
df.loc[df["distance"].between(5000,10000,inclusive="left"), "distancia_centro"] =  "Entre 5 y 10 km"
df.loc[df["distance"].between(10000,20000,inclusive="left"), "distancia_centro"] = "Entre 10 y 20 km"
df.loc[df["distance"].between(20000,30000,inclusive="left"), "distancia_centro"] = "Entre 20 y 30 km"
df.loc[df["distance"].between(30000,40000,inclusive="left"), "distancia_centro"] = "Entre 30 y 40 km"
df.loc[df["distance"].between(40000,50000,inclusive="both"), "distancia_centro"] = "Entre 40 y 50 km"
df.loc[df["distance"] > 50000, "distancia_centro"] = "Mas de 50 km"

In [108]:
df["distancia_centro"].value_counts()

distancia_centro
Entre 5 y 10 km     82
Entre 20 y 30 km    74
Entre 10 y 20 km    65
Entre 1 y 5 km      62
Entre 40 y 50 km    30
Entre 30 y 40 km    24
Menos de 1 km       12
Mas de 50 km         4
Name: count, dtype: int64

# Dropeamos distance

In [109]:
df.drop(columns="distance",inplace=True)

# Gestionar nulos variables categóricas
- Las booleanas las volvemos categóricas
- Los nans los pondremos en desconocido

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   propertyType      353 non-null    object 
 1   status            339 non-null    object 
 2   price             353 non-null    float64
 3   size              353 non-null    float64
 4   rooms             353 non-null    object 
 5   bathrooms         353 non-null    object 
 6   floor             290 non-null    object 
 7   neighborhood      174 non-null    object 
 8   district          301 non-null    object 
 9   province          353 non-null    object 
 10  municipality      353 non-null    object 
 11  exterior          353 non-null    bool   
 12  hasLift           335 non-null    object 
 13  hasPlan           339 non-null    object 
 14  has3DTour         339 non-null    object 
 15  has360            339 non-null    object 
 16  distancia_centro  353 non-null    object 
dt

### propertyType

In [111]:
df["propertyType"].isnull().sum()

np.int64(0)

### status

In [112]:
df["status"].isnull().sum()

np.int64(14)

In [113]:
df["status"]= df["status"].fillna("desconocido")
df["status"].unique()

array(['good', 'desconocido', 'newdevelopment', 'renew'], dtype=object)

In [114]:
df["status"].isnull().sum()

np.int64(0)

### floor

In [115]:
df["floor"].isnull().sum()

np.int64(63)

In [116]:
df["floor"]= df["floor"].fillna("desconocido")
df["floor"].unique()

array(['altos', 'bajos', 'desconocido'], dtype=object)

In [117]:
df["floor"].isnull().sum()

np.int64(0)

### neighborhood (Dropear Columna)
- Demasiados nulos, no tiene sentido

In [118]:
df["neighborhood"].isnull().sum()

np.int64(179)

In [119]:
df.drop(columns="neighborhood",inplace=True)

### district
- Tal vez especifique demasiado

In [120]:
df["district"].isnull().sum()

np.int64(52)

In [121]:
df["district"]= df["district"].fillna("desconocido")
df["district"].isnull().sum()

np.int64(0)

### province

In [122]:
df["province"].isnull().sum()

np.int64(0)

### municipality (Cambiado en Modelo 12)
- Vamos a reducir las opciones de la categoría
- Vamos a dividir por zonas en vez de municipios
    - Clase Alta
    - Clase Media-Alta
    - Clase Media
    - Clase Obrera

In [123]:
df["municipality"].isnull().sum()

np.int64(0)

In [124]:
clasificacion_municipios = {
    'Numancia de la Sagra': 'Clase Obrera',
    'Madrid': 'Clase Media-Alta',  # Varía según la zona
    'San Sebastián de los Reyes': 'Clase Alta',
    'Villamanrique de Tajo': 'Clase Obrera',
    'Recas': 'Clase Obrera',
    'Cedillo del Condado': 'Clase Obrera',
    'Rascafría': 'Clase Media',
    'Manzanares el Real': 'Clase Media',
    'Miraflores de la Sierra': 'Clase Media-Alta',
    'El Viso de San Juan': 'Clase Obrera',
    'Galapagar': 'Clase Media-Alta',
    'Arganda': 'Clase Media',
    'San Lorenzo de el Escorial': 'Clase Media-Alta',
    'Camarena': 'Clase Obrera',
    'Aldea del Fresno': 'Clase Obrera',
    'Aranjuez': 'Clase Media',
    'Villanueva del Pardillo': 'Clase Media-Alta',
    'Azuqueca de Henares': 'Clase Media',
    'El Espinar': 'Clase Media',
    'Las Rozas de Madrid': 'Clase Alta',
    'Guadalajara': 'Clase Media',
    'Illescas': 'Clase Obrera',
    'Navalcarnero': 'Clase Media',
    'Seseña': 'Clase Obrera',
    'Casarrubios del Monte': 'Clase Obrera',
    'Alcalá de Henares': 'Clase Media',
    'El Escorial': 'Clase Media-Alta',
    'Calypo Fado': 'Clase Obrera',
    'Leganés': 'Clase Media',
    'Coslada': 'Clase Media',
    'Torrejón de Ardoz': 'Clase Media',
    'Marchamalo': 'Clase Obrera',
    'Camarma de Esteruelas': 'Clase Media',
    'Alcorcón': 'Clase Media',
    'Pinto': 'Clase Media',
    'Pozo de Guadalajara': 'Clase Obrera',
    'Valdemoro': 'Clase Media',
    'Collado Villalba': 'Clase Media',
    'Getafe': 'Clase Media',
    'Paracuellos de Jarama': 'Clase Media-Alta',
    'El Molar': 'Clase Media',
    'Parla': 'Clase Obrera',
    'Tres Cantos': 'Clase Alta',
    'Yuncos': 'Clase Obrera',
    'Esquivias': 'Clase Obrera',
    'Quijorna': 'Clase Media',
    'Valdemorillo': 'Clase Media-Alta',
    'Yuncler': 'Clase Obrera',
    'Pedrezuela': 'Clase Media',
    'Daganzo de Arriba': 'Clase Media',
    'Yeles': 'Clase Obrera',
    'Guadarrama': 'Clase Media-Alta',
    'Ocaña': 'Clase Obrera',
    'Cobeña': 'Clase Media',
    'El Álamo': 'Clase Media',
    'Algete': 'Clase Media',
    'El Casar': 'Clase Obrera',
    'Rivas-Vaciamadrid': 'Clase Media-Alta',
    'Los Santos de la Humosa': 'Clase Media',
    'San Fernando de Henares': 'Clase Media',
    'Fuenlabrada': 'Clase Media',
    'Fuensalida': 'Clase Obrera',
    'Mataelpino': 'Clase Media',
    'Villa del Prado': 'Clase Media',
    'Mocejón': 'Clase Obrera',
    'Los Molinos': 'Clase Media',
    'Colmenar Viejo': 'Clase Media-Alta',
    'Móstoles': 'Clase Media',
    'Borox': 'Clase Obrera',
    'Navalafuente': 'Clase Media',
    'Meco': 'Clase Media',
    'Robledo de Chavela': 'Clase Media',
    'Campo Real': 'Clase Media',
    'Villaviciosa de Odón': 'Clase Alta',
    'San Ildefonso o la Granja': 'Clase Media',
    'Alameda de la Sagra': 'Clase Obrera',
    'Cabañas de la Sagra': 'Clase Obrera',
    'Las Navas del Marqués': 'Clase Media',
    'Villaseca de la Sagra': 'Clase Obrera',
    'Pozuelo de Alarcón': 'Clase Alta',
    'Yebes': 'Clase Obrera',
    'Bustarviejo': 'Clase Media',
    'Collado Mediano': 'Clase Media',
    'Chinchón': 'Clase Media',
    'Valmojado': 'Clase Obrera',
    'Alovera': 'Clase Media',
    'Colmenarejo': 'Clase Media-Alta',
    'Lominchar': 'Clase Obrera',
    'Loeches': 'Clase Media',
    'Sevilla la Nueva': 'Clase Media',
    'Serranillos del Valle': 'Clase Media',
    'Santa Cruz del Retamar': 'Clase Obrera',
    'Las Ventas de Retamosa': 'Clase Obrera',
    'Torrelaguna': 'Clase Media',
    'Villalbilla': 'Clase Media',
    'Alcobendas': 'Clase Alta',
}

# Asignar categorías al DataFrame
df["tipo_municipio"] = df["municipality"].map(clasificacion_municipios)

In [125]:
df["municipality"] = df["tipo_municipio"]
df.drop(columns="tipo_municipio", inplace=True)

In [126]:
df["municipality"].value_counts()

municipality
Clase Media-Alta    195
Clase Media         129
Clase Obrera         17
Clase Alta           12
Name: count, dtype: int64

###  exterior
- Cambiamos de booleano a categorica

In [127]:
df["exterior"].isnull().sum()

np.int64(0)

In [128]:
df["exterior"].unique()

array([ True, False])

In [129]:
df.loc[df["exterior"] == True, "exterior"] = "vista exterior"
df.loc[df["exterior"] == False, "exterior"] = "vista interior"
df["exterior"].unique()

array(['vista exterior', 'vista interior'], dtype=object)

### hasLift 
- Cambiamos de booleano a categorica

In [130]:
df["hasLift"].isnull().sum()

np.int64(18)

In [131]:
df["hasLift"].unique()

array([True, False, nan], dtype=object)

In [132]:
df.loc[df["hasLift"] == True, "hasLift"] = "tiene ascensor"
df.loc[df["hasLift"] == False, "hasLift"] = "no tiene ascensor"
df["hasLift"] = df["hasLift"].fillna("desconocido")
df["hasLift"].unique()

array(['tiene ascensor', 'no tiene ascensor', 'desconocido'], dtype=object)

In [133]:
df["hasLift"].isnull().sum()

np.int64(0)

### hasPlan
- Cambiamos de booleano a categorica


In [134]:
df["hasPlan"].isnull().sum()

np.int64(14)

In [135]:
df.loc[df["hasPlan"] == True, "hasPlan"] = "tiene planos"
df.loc[df["hasPlan"] == False, "hasPlan"] = "no tiene planos"
df["hasPlan"] = df["hasPlan"].fillna("desconocido")
df["hasPlan"].unique()

array(['no tiene planos', 'tiene planos', 'desconocido'], dtype=object)

In [136]:
df["hasPlan"].isnull().sum()

np.int64(0)

### has3DTour
- Cambiamos de booleano a categorica


In [137]:
df["has3DTour"].isnull().sum()

np.int64(14)

In [138]:
df.loc[df["has3DTour"] == True, "has3DTour"] = "tiene tour 3D"
df.loc[df["has3DTour"] == False, "has3DTour"] = "no tiene tour 3D"
df["has3DTour"] = df["has3DTour"].fillna("desconocido")
df["has3DTour"].unique()

array(['no tiene tour 3D', 'desconocido', 'tiene tour 3D'], dtype=object)

In [139]:
df["has3DTour"].isnull().sum()

np.int64(0)

### has360
- Cambiamos de booleano a categorica

In [140]:
df["has360"].isnull().sum()

np.int64(14)

In [141]:
c = df["has360"].isnull() == True
df[c]


,propertyType,status,price,size,rooms,bathrooms,floor,district,province,municipality,exterior,hasLift,hasPlan,has3DTour,has360,distancia_centro
4,studio,desconocido,684.00,45.00,sin habitaciones,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 10 y 20 km
51,flat,desconocido,700.00,50.00,1 habitacion,2 aseos,desconocido,desconocido,Madrid,Clase Media,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 30 y 40 km
52,duplex,desconocido,750.00,35.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 1 y 5 km
106,flat,desconocido,600.00,55.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Clase Media,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 20 y 30 km
121,flat,desconocido,630.00,54.00,1 habitacion,2 aseos,desconocido,desconocido,Madrid,Clase Media,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 30 y 40 km
137,studio,desconocido,700.00,23.00,sin habitaciones,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 1 y 5 km
147,duplex,desconocido,600.00,80.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista interior,desconocido,desconocido,desconocido,NaN,Entre 5 y 10 km
148,duplex,desconocido,600.00,80.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista interior,desconocido,desconocido,desconocido,NaN,Entre 5 y 10 km
176,flat,desconocido,675.00,35.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Clase Media-Alta,vista interior,desconocido,desconocido,desconocido,NaN,Entre 1 y 5 km
183,flat,desconocido,750.00,67.00,3 habitaciones,1 aseo,desconocido,desconocido,Madrid,Clase Media,vista exterior,desconocido,desconocido,desconocido,NaN,Entre 10 y 20 km


#### Habrá que ver si estos datos merece la pena tenerlos, hay muchas cosas desconocidas

In [142]:
df.loc[df["has360"] == True, "has360"] = "tiene fotos 360"
df.loc[df["has360"] == False, "has360"] = "no tiene fotos 360"
df["has360"] = df["has360"].fillna("desconocido")
df["has360"].unique()

array(['no tiene fotos 360', 'tiene fotos 360', 'desconocido'],
      dtype=object)

In [143]:
df["has360"].isnull().sum()

np.int64(0)

### distancia_centro

In [144]:
df["distancia_centro"].isnull().sum()

np.int64(0)

# Gestión nulos numéricas
- Sin tocar la VR (Precio).
- Rellenaremos con IterativeImputer y RandomForest
    - Las relaciones son complejas

In [145]:
df.isnull().sum()

propertyType        0
status              0
price               0
size                0
rooms               0
bathrooms           0
floor               0
district            0
province            0
municipality        0
exterior            0
hasLift             0
hasPlan             0
has3DTour           0
has360              0
distancia_centro    0
dtype: int64

# Modelo 7 eliminar province

In [146]:
df.drop(columns="province",inplace=True)

# Modelo 8 Eliminar district y has3DTour

In [147]:
df.drop(columns=["district","has3DTour"],inplace=True)

# Modelo 9 Eliminar status

In [148]:
df["status"].value_counts()

status
good              324
desconocido        14
newdevelopment     13
renew               2
Name: count, dtype: int64

In [149]:
df.drop(columns="status",inplace=True)

# Modelo 11 Eliminar distancia_centro


In [150]:
df["distancia_centro"].value_counts()

distancia_centro
Entre 5 y 10 km     82
Entre 20 y 30 km    74
Entre 10 y 20 km    65
Entre 1 y 5 km      62
Entre 40 y 50 km    30
Entre 30 y 40 km    24
Menos de 1 km       12
Mas de 50 km         4
Name: count, dtype: int64

In [151]:
#df.drop(columns="distancia_centro",inplace=True)

# Modelo 13 Eliminar has360 y hasPlan
- Considero que el usuario no tiene porque querer saber esto


In [152]:
df["has360"].value_counts()

has360
no tiene fotos 360    253
tiene fotos 360        86
desconocido            14
Name: count, dtype: int64

In [153]:
df["hasPlan"].value_counts()

hasPlan
no tiene planos    221
tiene planos       118
desconocido         14
Name: count, dtype: int64

In [154]:
df.drop(columns=["has360","hasPlan"],inplace=True)

In [155]:
df["propertyType"].value_counts()

propertyType
flat            277
studio           48
duplex           14
penthouse        10
chalet            3
countryHouse      1
Name: count, dtype: int64

In [156]:
c1 = df["propertyType"] != "flat"
c2 = df["propertyType"] != "studio"

# Modelo 15 quitar "exterior"

In [157]:
df.drop(columns="exterior",inplace=True)

# Modelo 16 
- Usando property, se rellenan desconocidos de:
    - hasLift
    - floor



In [158]:
df["propertyType"].value_counts()

propertyType
flat            277
studio           48
duplex           14
penthouse        10
chalet            3
countryHouse      1
Name: count, dtype: int64

### que hago
- Todo lo que sea:
    - duplex
    - chalet
    - countryhouse
- No tienen ascensor y no tienen "floor" que se pondrá como "bajos"

- Los penthouse son áticos de lujo por lo que:
    - floor = altos
    - hasLift = si, porque es de lujo

In [159]:
tipos = ["duplex","chalet","countryHouse"]
df.loc[df["propertyType"].isin(tipos),"hasLift"] = "no tiene ascensor"
df.loc[df["propertyType"].isin(tipos),"floor"] = "bajos"
df.loc[df["propertyType"] == "penthouse","hasLift"] = "tiene ascensor"
df.loc[df["propertyType"] == "penthouse","floor"] = "altos"

In [160]:
df["floor"].value_counts()

floor
altos          155
bajos          144
desconocido     54
Name: count, dtype: int64

In [161]:
df["hasLift"].value_counts()

hasLift
tiene ascensor       178
no tiene ascensor    164
desconocido           11
Name: count, dtype: int64

### Eliminar propertyType y distancia_centro (Modelo 16)

In [162]:
df.drop(columns=["propertyType","distancia_centro"],inplace=True)

# Guardamos

In [163]:
df.to_csv("../../datos/03_api_rent_sin_nulos.csv",index=False)